In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 8
year = 2024
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.head()

,started_at,start_station_id,rideable_type,member_casual
0,2024-08-13 21:12:42.989,6072,1,1
1,2024-08-14 17:12:02.513,6072,0,1
2,2024-08-02 21:59:31.775,6072,1,1
3,2024-08-07 18:43:33.357,6072,1,1
4,2024-08-02 08:03:39.557,6072,0,1


In [26]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136344 entries, 0 to 136343
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   started_at        136344 non-null  datetime64[ns]
 1   start_station_id  136344 non-null  int64         
 2   rideable_type     136344 non-null  int64         
 3   member_casual     136344 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 4.2 MB


In [27]:
rides["pickup_hour"] = rides["started_at"].dt.floor('6H')
rides.head()

,started_at,start_station_id,rideable_type,member_casual,pickup_hour
0,2024-08-13 21:12:42.989,6072,1,1,2024-08-13 18:00:00
1,2024-08-14 17:12:02.513,6072,0,1,2024-08-14 12:00:00
2,2024-08-02 21:59:31.775,6072,1,1,2024-08-02 18:00:00
3,2024-08-07 18:43:33.357,6072,1,1,2024-08-07 18:00:00
4,2024-08-02 08:03:39.557,6072,0,1,2024-08-02 06:00:00


In [28]:
agg_rides = rides.groupby(["pickup_hour", "start_station_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.head()

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
1,2024-07-31 18:00:00,5779,8
2,2024-07-31 18:00:00,5788,9
3,2024-07-31 18:00:00,5905,6
4,2024-07-31 18:00:00,6072,3


In [29]:
# get all the hours
hour_col = "pickup_hour"
full_hours = pd.date_range(
    start=agg_rides[hour_col].min(),
    end=agg_rides[hour_col].max(),
    freq="6H"
)

full_hours

DatetimeIndex(['2024-07-31 18:00:00', '2024-08-01 00:00:00',
               '2024-08-01 06:00:00', '2024-08-01 12:00:00',
               '2024-08-01 18:00:00', '2024-08-02 00:00:00',
               '2024-08-02 06:00:00', '2024-08-02 12:00:00',
               '2024-08-02 18:00:00', '2024-08-03 00:00:00',
               ...
               '2024-08-29 12:00:00', '2024-08-29 18:00:00',
               '2024-08-30 00:00:00', '2024-08-30 06:00:00',
               '2024-08-30 12:00:00', '2024-08-30 18:00:00',
               '2024-08-31 00:00:00', '2024-08-31 06:00:00',
               '2024-08-31 12:00:00', '2024-08-31 18:00:00'],
              dtype='datetime64[ns]', length=125, freq='6H')

In [30]:
location_col = "start_station_id"
all_locations = agg_rides[location_col].unique()
all_locations

array([5626, 5779, 5788, 5905, 6072], dtype=int64)

In [31]:
# Create a list to store the complete data
complete_data = []
rides_col = "rides"
# Iterate over all combinations of hours and locations
for hour in full_hours:
    for location in all_locations:
        # Check if the combination exists in the original DataFrame
        subset = agg_rides[(agg_rides[hour_col] == hour) & (agg_rides[location_col] == location)]
        if not subset.empty:
            # If the combination exists, append the row
            complete_data.append(subset.iloc[0].to_dict())
        else:
            # If the combination is missing, append a row with 0 rides
            complete_data.append({hour_col: hour, location_col: location, rides_col: 0})

df_complete = pd.DataFrame(complete_data)
df_complete

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
1,2024-07-31 18:00:00,5779,8
2,2024-07-31 18:00:00,5788,9
3,2024-07-31 18:00:00,5905,6
4,2024-07-31 18:00:00,6072,3
...,...,...,...
620,2024-08-31 18:00:00,5626,209
621,2024-08-31 18:00:00,5779,247
622,2024-08-31 18:00:00,5788,246
623,2024-08-31 18:00:00,5905,195


In [32]:
df_complete.sort_values(["start_station_id", "pickup_hour"])

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
5,2024-08-01 00:00:00,5626,47
10,2024-08-01 06:00:00,5626,282
15,2024-08-01 12:00:00,5626,227
20,2024-08-01 18:00:00,5626,303
...,...,...,...
604,2024-08-30 18:00:00,6072,193
609,2024-08-31 00:00:00,6072,56
614,2024-08-31 06:00:00,6072,97
619,2024-08-31 12:00:00,6072,228


In [33]:
# Create full combinations of hours and locations
full_combinations = pd.DataFrame(
    [(hour, location) for hour in full_hours for location in all_locations],
    columns=["pickup_hour", "start_station_id"]
)

full_combinations

,pickup_hour,start_station_id
0,2024-07-31 18:00:00,5626
1,2024-07-31 18:00:00,5779
2,2024-07-31 18:00:00,5788
3,2024-07-31 18:00:00,5905
4,2024-07-31 18:00:00,6072
...,...,...
620,2024-08-31 18:00:00,5626
621,2024-08-31 18:00:00,5779
622,2024-08-31 18:00:00,5788
623,2024-08-31 18:00:00,5905


In [34]:
merged_df = pd.merge(
    full_combinations,
    agg_rides,
    on=["pickup_hour", "start_station_id"],
    how="left"
)
merged_df

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
1,2024-07-31 18:00:00,5779,8
2,2024-07-31 18:00:00,5788,9
3,2024-07-31 18:00:00,5905,6
4,2024-07-31 18:00:00,6072,3
...,...,...,...
620,2024-08-31 18:00:00,5626,209
621,2024-08-31 18:00:00,5779,247
622,2024-08-31 18:00:00,5788,246
623,2024-08-31 18:00:00,5905,195


In [35]:
merged_df["rides"] = merged_df["rides"].fillna(0).astype(int)
merged_df

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
1,2024-07-31 18:00:00,5779,8
2,2024-07-31 18:00:00,5788,9
3,2024-07-31 18:00:00,5905,6
4,2024-07-31 18:00:00,6072,3
...,...,...,...
620,2024-08-31 18:00:00,5626,209
621,2024-08-31 18:00:00,5779,247
622,2024-08-31 18:00:00,5788,246
623,2024-08-31 18:00:00,5905,195


In [36]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col, freq="6H"):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq=freq
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [37]:
agg_rides

,pickup_hour,start_station_id,rides
0,2024-07-31 18:00:00,5626,3
1,2024-07-31 18:00:00,5779,8
2,2024-07-31 18:00:00,5788,9
3,2024-07-31 18:00:00,5905,6
4,2024-07-31 18:00:00,6072,3
...,...,...,...
620,2024-08-31 18:00:00,5626,209
621,2024-08-31 18:00:00,5779,247
622,2024-08-31 18:00:00,5788,246
623,2024-08-31 18:00:00,5905,195


In [38]:
hour_col = "pickup_hour"
location_col = "start_station_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["start_station_id", "pickup_hour"]).reset_index(drop=True)

In [39]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.start_station_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="start_station_id",
        template="none"
    )

    fig.show()

In [40]:
plot_rides(agg_data_filled, locations=[5626, 5779, 6072])

In [41]:
month = 8
year = 2024
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

agg_data_filled.to_parquet(path)